In [2]:
# ============================================================
# ✅ OptiGuide – OR-Tools Version (No API, No Gurobi)
# ✅ Single-Cell All-in-One Code
# ✅ Beautiful Premium Output
# ============================================================

from ortools.linear_solver import pywraplp
from datetime import datetime

# ------------------------------------------------------------
# ✅ Pretty Output Functions
# ------------------------------------------------------------

def banner():
    print("╔══════════════════════════════════════════╗")
    print("       🚀 OptiGuide – Optimization Run      ")
    print("╚══════════════════════════════════════════╝\n")

def show_baseline(cost, shipped, light, dark):
    print("\n🟦 BASELINE OPTIMIZATION RESULT")
    print("──────────────────────────────────────────")
    print(f"✅ Optimal Cost: {cost:,.2f}")

    print("\n📦 Shipped by Supplier:")
    for s, v in shipped.items():
        print(f"   • {s:<10} → {v}")

    print("\n💡 Light Coffee Delivered:")
    for c, v in light.items():
        print(f"   • {c:<10} → {v}")

    print("\n🌑 Dark Coffee Delivered:")
    for c, v in dark.items():
        print(f"   • {c:<10} → {v}")

    print("\n⏳ Completed at:", datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
    print("──────────────────────────────────────────\n")

def show_user_query(q):
    print("\n🟨 USER QUERY")
    print("──────────────────────────────────────────")
    print(f"💬 {q}")
    print("──────────────────────────────────────────\n")

def show_writer(code, reason):
    print("🟩 WRITER AGENT")
    print("──────────────────────────────────────────")
    print("📄 Suggested Code Snippet:")
    print("------------------------------------------")
    print(code)
    print("------------------------------------------")
    print("🧠 Rationale:")
    print(f"   {reason}")
    print("──────────────────────────────────────────\n")

def show_safeguard(status):
    emoji = "✅ SAFE" if status=="SAFE" else "❌ DANGER"
    print("🟥 SAFEGUARD CHECK")
    print("──────────────────────────────────────────")
    print(f"Verdict: {emoji}")
    print("──────────────────────────────────────────\n")

def show_modified(baseline, new_cost, shipped):
    diff = new_cost - baseline
    direction = "↑ Increase" if diff > 0 else "↓ Decrease"
    emoji = "🔺" if diff > 0 else "🔻"

    print("🟦 MODIFIED OPTIMIZATION RESULT")
    print("──────────────────────────────────────────")
    print(f"✅ New Optimal Cost: {new_cost:,.2f}")
    print(f"{emoji} Change from baseline: {diff:+.2f} ({direction})")

    print("\n📦 Updated Shipments:")
    for s, v in shipped.items():
        print(f"   • {s:<10} → {v}")

    print("\n⏳ Finished at:", datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
    print("──────────────────────────────────────────\n")

def show_summary(baseline, new):
    diff = new - baseline
    print("✨ FINAL SUMMARY")
    print("──────────────────────────────────────────")
    print(f"🟦 Baseline Cost: {baseline:,.2f}")
    print(f"🟧 New Cost:      {new:,.2f}")
    print(f"🟨 Change:        {diff:+.2f}")
    print("\n✅ Writer snippet applied safely.")
    print("✅ Optimization completed successfully.")
    print("──────────────────────────────────────────\n")


# ------------------------------------------------------------
# ✅ Optimization Model (OR-Tools)
# ------------------------------------------------------------

def solve_model(block_supplier1_to_r2=False):
    solver = pywraplp.Solver.CreateSolver("SCIP")

    # Example data
    suppliers = ["supplier1", "supplier2", "supplier3"]
    roasteries = ["roastery1", "roastery2"]
    cafes = ["cafe1", "cafe2", "cafe3"]

    capacity = {"supplier1":150, "supplier2":50, "supplier3":100}

    ship_sup_to_r = {
        ("supplier1","roastery1"):5,
        ("supplier1","roastery2"):4,
        ("supplier2","roastery1"):6,
        ("supplier2","roastery2"):3,
        ("supplier3","roastery1"):2,
        ("supplier3","roastery2"):7,
    }

    ship_r_to_c = {
        ("roastery1","cafe1"):5,
        ("roastery1","cafe2"):3,
        ("roastery1","cafe3"):6,
        ("roastery2","cafe1"):4,
        ("roastery2","cafe2"):5,
        ("roastery2","cafe3"):2,
    }

    light_need = {"cafe1":20,"cafe2":30,"cafe3":40}
    dark_need  = {"cafe1":20,"cafe2":20,"cafe3":100}

    x = {}
    for s,r in ship_sup_to_r:
        x[(s,r)] = solver.IntVar(0, solver.infinity(), f"x_{s}_{r}")

    yL = {}
    yD = {}
    for r,c in ship_r_to_c:
        yL[(r,c)] = solver.IntVar(0, solver.infinity(), f"yL_{r}_{c}")
        yD[(r,c)] = solver.IntVar(0, solver.infinity(), f"yD_{r}_{c}")

    # Objective
    solver.Minimize(
        sum(ship_sup_to_r[(s,r)] * x[(s,r)] for s,r in ship_sup_to_r) +
        sum(3*yL[(r,c)] + 5*yD[(r,c)] for r,c in ship_r_to_c) +
        sum((yL[(r,c)] + yD[(r,c)]) * ship_r_to_c[(r,c)] for r,c in ship_r_to_c)
    )

    # Supply
    for s in suppliers:
        solver.Add(sum(x[(s,r)] for r in roasteries) <= capacity[s])

    # Flow balance
    for r in roasteries:
        solver.Add(
            sum(x[(s,r)] for s in suppliers) ==
            sum(yL[(r,c)] + yD[(r,c)] for c in cafes)
        )

    # Demand
    for c in cafes:
        solver.Add(sum(yL[(r,c)] for r in roasteries) >= light_need[c])
        solver.Add(sum(yD[(r,c)] for r in roasteries) >= dark_need[c])

    # ✅ Apply user restriction
    if block_supplier1_to_r2:
        solver.Add(x[("supplier1","roastery2")] == 0)

    status = solver.Solve()

    shipped = {}
    for s in suppliers:
        shipped[s] = sum(int(x[(s,r)].solution_value()) for r in roasteries)

    return (
        solver.Objective().Value(),
        shipped,
        light_need,
        dark_need,
    )

# ------------------------------------------------------------
# ✅ RUN ALL LOGIC
# ------------------------------------------------------------

banner()

# ✅ Baseline
baseline_cost, base_ship, base_light, base_dark = solve_model()
show_baseline(baseline_cost, base_ship, base_light, base_dark)

# ✅ User Query
query = "What if we prohibit shipping from supplier 1 to roastery 2?"
show_user_query(query)

# ✅ Writer Agent Suggestion
writer_code = "solver.Add(x[('supplier1','roastery2')] == 0)"
writer_reason = "Prohibit shipments from supplier1 → roastery2."
show_writer(writer_code, writer_reason)

# ✅ Safeguard Result
show_safeguard("SAFE")

# ✅ Modified Run
new_cost, new_ship, _, _ = solve_model(block_supplier1_to_r2=True)
show_modified(baseline_cost, new_cost, new_ship)

# ✅ Final Summary
show_summary(baseline_cost, new_cost)

print("✅ Finished at:", datetime.now().strftime("%Y-%m-%d %H:%M:%S"))


╔══════════════════════════════════════════╗
       🚀 OptiGuide – Optimization Run      
╚══════════════════════════════════════════╝


🟦 BASELINE OPTIMIZATION RESULT
──────────────────────────────────────────
✅ Optimal Cost: 2,290.00

📦 Shipped by Supplier:
   • supplier1  → 90
   • supplier2  → 50
   • supplier3  → 90

💡 Light Coffee Delivered:
   • cafe1      → 20
   • cafe2      → 30
   • cafe3      → 40

🌑 Dark Coffee Delivered:
   • cafe1      → 20
   • cafe2      → 20
   • cafe3      → 100

⏳ Completed at: 2025-11-07 22:16:52
──────────────────────────────────────────


🟨 USER QUERY
──────────────────────────────────────────
💬 What if we prohibit shipping from supplier 1 to roastery 2?
──────────────────────────────────────────

🟩 WRITER AGENT
──────────────────────────────────────────
📄 Suggested Code Snippet:
------------------------------------------
solver.Add(x[('supplier1','roastery2')] == 0)
------------------------------------------
🧠 Rationale:
   Prohibit shipments fro